In [1]:
# Import the required libraries
import torch
from PIL import Image
import torchvision.transforms as T
import os
import cv2
# assign directory
dataset = []
data = dict()
directory = ['/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID/images', '/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/images', '/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia/images', '/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Lung_Opacity/images']
labels = ['COVID','Normal','Pneumonia','Bacterial infection']
cnt = 0
# iterate over files in
# that directory
ids = 0
for dirs in directory:
    cnt = 0
    for filename in os.listdir(dirs):
        f = os.path.join(dirs, filename)
        
        if os.path.isfile(f):
            trans_comp = T.Compose([
                         T.Resize([224,224]),
                         T.Grayscale(num_output_channels=1),
                         T.ToTensor()
                        ])
            im = Image.open(f)
             
            tensor = trans_comp(im)
            print(f'{cnt}-{tensor.shape}');
            dataset.append([tensor,ids])
            cnt+=1
        
        
             
     
    ids+=1


0-torch.Size([1, 224, 224])
1-torch.Size([1, 224, 224])
2-torch.Size([1, 224, 224])
3-torch.Size([1, 224, 224])
4-torch.Size([1, 224, 224])
5-torch.Size([1, 224, 224])
6-torch.Size([1, 224, 224])
7-torch.Size([1, 224, 224])
8-torch.Size([1, 224, 224])
9-torch.Size([1, 224, 224])
10-torch.Size([1, 224, 224])
11-torch.Size([1, 224, 224])
12-torch.Size([1, 224, 224])
13-torch.Size([1, 224, 224])
14-torch.Size([1, 224, 224])
15-torch.Size([1, 224, 224])
16-torch.Size([1, 224, 224])
17-torch.Size([1, 224, 224])
18-torch.Size([1, 224, 224])
19-torch.Size([1, 224, 224])
20-torch.Size([1, 224, 224])
21-torch.Size([1, 224, 224])
22-torch.Size([1, 224, 224])
23-torch.Size([1, 224, 224])
24-torch.Size([1, 224, 224])
25-torch.Size([1, 224, 224])
26-torch.Size([1, 224, 224])
27-torch.Size([1, 224, 224])
28-torch.Size([1, 224, 224])
29-torch.Size([1, 224, 224])
30-torch.Size([1, 224, 224])
31-torch.Size([1, 224, 224])
32-torch.Size([1, 224, 224])
33-torch.Size([1, 224, 224])
34-torch.Size([1, 224, 2

In [ ]:
import matplotlib.pyplot as plt

def show_bar(D):

    plt.bar(range(len(D)), list(D.values()), align='center')
    plt.xticks(range(len(D)), list(D.keys()))
    # # for python 2.x:
    # plt.bar(range(len(D)), D.values(), align='center')  # python 2.x
    # plt.xticks(range(len(D)), D.keys())  # in python 2.x

    plt.show()

# COVID-19 radiography dataset

In [10]:
import numpy as np
print(len(dataset))
x = []
y = []
for i in dataset:
    x.append(i[0])
    y.append(i[1]) 
 

27228


# Imbalanced dataset training

In [12]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [13]:
xtr, xts, ytr, yts = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
del x
del y

In [17]:
print(f'test {len(xts)}--train {len(xtr)}')

test 5446--train 21782


In [10]:
import torch
xtr = xtr.permute(0,3,1,2)
xts = xts.permute(0,3,1,2)

AttributeError: 'list' object has no attribute 'permute'

In [18]:
xts = torch.stack((xts))
xtr = torch.stack((xtr))
 

In [22]:
ytr = torch.FloatTensor(ytr)
yts = torch.FloatTensor(yts)

In [26]:
torch.save(xtr, '/kaggle/working/trainx.pt')
torch.save(xts, '/kaggle/working/testx.pt')
torch.save(ytr, '/kaggle/working/trainy.pt')
torch.save(yts, '/kaggle/working/testy.pt')

In [ ]:
import torch
from torch import *
my_dataset = torch.utils.data.TensorDataset(xtr,ytr) # create your datset
my_dataloader = torch.utils.data.DataLoader(my_dataset,batch_size=4,
                                          shuffle=True, num_workers=2)

my_dataset = torch.utils.data.TensorDataset(xts,yts) # create your datset
my_tdataloader = torch.utils.data.DataLoader(my_dataset,batch_size=32)

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self, num_classes=4):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)

        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5) 
        self.fc1 = nn.Linear(16 * 5 * 5, 120)# 5x5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

  

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
device

In [ ]:
# Instantiate the model 
model = CNNModel() 
  
# Move the model to the GPU if available 
model.to(device) 

In [ ]:
import torch.optim as optim 

In [ ]:
# Define a loss function and optimizer 
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
batch_size=4
num_epochs=10

losses = [] 
accuracies = [] 
  
# Train the model 
for epoch in range(1,num_epochs): 
    y_pred = []
    for i, (images, labels) in enumerate(my_dataloader): 
# Forward pass 
        images=images.to(device) 
        labels=labels.to(device) 
        outputs = model(images)
        loss = criterion(outputs, labels) 
        y_pred.append(outputs)
        # Backward pass and optimization 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 

        _, predicted = torch.max(outputs.data, 1) 
    acc = (predicted == labels).sum().item() / labels.size(0) 
    accuracies.append(acc) 
    losses.append(loss.item()) 
        
    # Evaluate the model on the validation set 	 
    print('Epoch [{}/{}],Loss:{:.4f} ,Accuracy:{:.2f} '.format( 
        epoch+1, num_epochs, loss.item(),acc))

# Pneumonia merge

In [2]:
 
directory = ['/kaggle/input/labeled-chest-xray-images/chest_xray/test/PNEUMONIA',
            '/kaggle/input/labeled-chest-xray-images/chest_xray/train/PNEUMONIA']
cnt = 0
# iterate over files in
# that directory
ids = 0
for dirs in directory:
    cnt = 0
    for filename in os.listdir(dirs):
        f = os.path.join(dirs, filename)

        if os.path.isfile(f):
            trans_comp = T.Compose([
                         T.Resize([224,224]),
                         T.Grayscale(num_output_channels=1),
                         T.ToTensor()
                        ])
            im = Image.open(f)
             
            tensor = trans_comp(im)
            print(f'{cnt}-{tensor.shape}');
            dataset.append([tensor,2])
    
     
 

0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224

In [ ]:
D = data

plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()))
# # for python 2.x:
# plt.bar(range(len(D)), D.values(), align='center')  # python 2.x
# plt.xticks(range(len(D)), D.keys())  # in python 2.x

plt.show()

# Covid Merge

In [3]:
 
directory = ['/kaggle/input/covid-xray-dataset/DATA/DATA/1']
cnt = 0
 
for dirs in directory:
    cnt = 0
    for filename in os.listdir(dirs):
        f = os.path.join(dirs, filename)

        if os.path.isfile(f):
            trans_comp = T.Compose([
                         T.Resize([224,224]),
                         T.Grayscale(num_output_channels=1),
                         T.ToTensor()
                        ])
            im = Image.open(f)
             
            tensor = trans_comp(im)
            print(f'{cnt}-{tensor.shape}');
            dataset.append([tensor,0])

0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224, 224])
0-torch.Size([1, 224

In [9]:
len(dataset)

1